# Введение

В этом задании Вы продолжите работать с данными из семинара [Articles Sharing and Reading from CI&T Deskdrop](https://www.kaggle.com/gspmoreira/articles-sharing-reading-from-cit-deskdrop).

# Загрузка и предобработка данных

In [1]:
import math

import numpy as np
import pandas as pd

Загрузим данные и проведем предобработку данных как на семинаре.

In [3]:
!mkdir -p ~/.kaggle # creating .kaggle folder where the key should be placed
!cp kaggle.json ~/.kaggle/ # move the key to the folder

In [4]:
!kaggle datasets download -d gspmoreira/articles-sharing-reading-from-cit-deskdrop
!unzip articles-sharing-reading-from-cit-deskdrop.zip -d articles

 98% 8.00M/8.20M [00:01<00:00, 12.8MB/s]
100% 8.20M/8.20M [00:01<00:00, 7.97MB/s]
Archive:  articles-sharing-reading-from-cit-deskdrop.zip
  inflating: articles/shared_articles.csv  
  inflating: articles/users_interactions.csv  


In [5]:
articles_df = pd.read_csv("articles/shared_articles.csv")
articles_df = articles_df[articles_df["eventType"] == "CONTENT SHARED"]
articles_df.head(2)

,timestamp,eventType,contentId,authorPersonId,authorSessionId,authorUserAgent,authorRegion,authorCountry,contentType,url,title,text,lang
1,1459193988,CONTENT SHARED,-4110354420726924665,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://www.nytimes.com/2016/03/28/business/dea...,"Ethereum, a Virtual Currency, Enables Transact...",All of this work is still very early. The firs...,en
2,1459194146,CONTENT SHARED,-7292285110016212249,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://cointelegraph.com/news/bitcoin-future-w...,Bitcoin Future: When GBPcoin of Branson Wins O...,The alarm clock wakes me at 8:00 with stream o...,en


In [6]:
interactions_df = pd.read_csv("articles/users_interactions.csv")
interactions_df.head(2)

,timestamp,eventType,contentId,personId,sessionId,userAgent,userRegion,userCountry
0,1465413032,VIEW,-3499919498720038879,-8845298781299428018,1264196770339959068,NaN,NaN,NaN
1,1465412560,VIEW,8890720798209849691,-1032019229384696495,3621737643587579081,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2...,NY,US


In [7]:
interactions_df.personId = interactions_df.personId.astype(str)
interactions_df.contentId = interactions_df.contentId.astype(str)
articles_df.contentId = articles_df.contentId.astype(str)

In [8]:
# зададим словарь определяющий силу взаимодействия
event_type_strength = {
    "VIEW": 1.0,
    "LIKE": 2.0,
    "BOOKMARK": 2.5,
    "FOLLOW": 3.0,
    "COMMENT CREATED": 4.0,
}

interactions_df["eventStrength"] = interactions_df.eventType.apply(
    lambda x: event_type_strength[x]
)

Оставляем только тех пользователей, которые произамодействовали более чем с пятью статьями.

In [9]:
users_interactions_count_df = (
    interactions_df.groupby(["personId", "contentId"])
    .first()
    .reset_index()
    .groupby("personId")
    .size()
)
print("# users:", len(users_interactions_count_df))

users_with_enough_interactions_df = users_interactions_count_df[
    users_interactions_count_df >= 5
].reset_index()[["personId"]]
print("# users with at least 5 interactions:", len(users_with_enough_interactions_df))

# users: 1895
# users with at least 5 interactions: 1140


Оставляем только те взаимодействия, которые относятся к отфильтрованным пользователям.

In [10]:
interactions_from_selected_users_df = interactions_df.loc[
    np.in1d(interactions_df.personId, users_with_enough_interactions_df)
]

In [11]:
print(f"# interactions before: {interactions_df.shape}")
print(f"# interactions after: {interactions_from_selected_users_df.shape}")

# interactions before: (72312, 9)
# interactions after: (69868, 9)


Объединяем все взаимодействия пользователя по каждой статье и сглаживаем полученный результат, взяв от него логарифм.

In [12]:
def smooth_user_preference(x):
    return math.log(1 + x, 2)


interactions_full_df = (
    interactions_from_selected_users_df.groupby(["personId", "contentId"])
    .eventStrength.sum()
    .apply(smooth_user_preference)
    .reset_index()
    .set_index(["personId", "contentId"])
)
interactions_full_df["last_timestamp"] = interactions_from_selected_users_df.groupby(
    ["personId", "contentId"]
)["timestamp"].last()

interactions_full_df = interactions_full_df.reset_index()
interactions_full_df.head(5)

,personId,contentId,eventStrength,last_timestamp
0,-1007001694607905623,-5065077552540450930,1.000000,1470395911
1,-1007001694607905623,-6623581327558800021,1.000000,1487240080
2,-1007001694607905623,-793729620925729327,1.000000,1472834892
3,-1007001694607905623,1469580151036142903,1.000000,1487240062
4,-1007001694607905623,7270966256391553686,1.584963,1485994324


Разобьём выборку на обучение и контроль по времени.

In [13]:
from sklearn.model_selection import train_test_split

split_ts = 1475519530
interactions_train_df = interactions_full_df.loc[
    interactions_full_df.last_timestamp < split_ts
].copy()
interactions_test_df = interactions_full_df.loc[
    interactions_full_df.last_timestamp >= split_ts
].copy()

print(f"# interactions on Train set: {len(interactions_train_df)}")
print(f"# interactions on Test set: {len(interactions_test_df)}")

interactions_train_df

# interactions on Train set: 29329
# interactions on Test set: 9777


,personId,contentId,eventStrength,last_timestamp
0,-1007001694607905623,-5065077552540450930,1.0,1470395911
2,-1007001694607905623,-793729620925729327,1.0,1472834892
6,-1032019229384696495,-1006791494035379303,1.0,1469129122
7,-1032019229384696495,-1039912738963181810,1.0,1459376415
8,-1032019229384696495,-1081723567492738167,2.0,1464054093
...,...,...,...,...
39099,997469202936578234,9112765177685685246,2.0,1472479493
39100,998688566268269815,-1255189867397298842,1.0,1474567164
39101,998688566268269815,-401664538366009049,1.0,1474567449
39103,998688566268269815,6881796783400625893,1.0,1474567675


Для удобства подсчёта качества запишем данные в формате, где строка соответствует пользователю, а столбцы будут истинными метками и предсказаниями в виде списков.

In [14]:
interactions = (
    interactions_train_df.groupby("personId")["contentId"]
    .agg(lambda x: list(x))
    .reset_index()
    .rename(columns={"contentId": "true_train"})
    .set_index("personId")
)

interactions["true_test"] = interactions_test_df.groupby("personId")["contentId"].agg(
    lambda x: list(x)
)

# заполнение пропусков пустыми списками
interactions.loc[pd.isnull(interactions.true_test), "true_test"] = [
    ""
    for x in range(
        len(interactions.loc[pd.isnull(interactions.true_test), "true_test"])
    )
]

interactions.head(1)

,true_train,true_test
personId,,
-1007001694607905623,"[-5065077552540450930, -793729620925729327]","[-6623581327558800021, 1469580151036142903, 72..."


# Библиотека LightFM

Для рекомендации Вы будете пользоваться библиотекой [LightFM](https://making.lyst.com/lightfm/docs/home.html), в которой реализованы популярные алгоритмы. Для оценивания качества рекомендации, как и на семинаре, будем пользоваться метрикой *precision@10*.

In [15]:
!pip install lightfm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.4/316.4 KB 6.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lightfm: filename=lightfm-1.17-cp39-cp39-linux_x86_64.whl size=889520 sha256=72d6aeead6027ce4cd3a00b57bc5eb3df4d254491b5e7b6ded89d6af6a464e0d
  Stored in directory: /root/.cache/pip/wheels/d8/65/93/6ac8180274dc2e8f86ff326be62da1dfa55dc158fd45faba7d
Successfully built lightfm


In [16]:
from lightfm import LightFM
from lightfm.evaluation import precision_at_k

## Задание 1 (1.5 балла)

Модели в LightFM работают с разреженными матрицами. Создайте разреженные матрицы `data_train` и `data_test` (размером количество пользователей на количество статей), такие что на пересечении строки пользователя и столбца статьи стоит сила их взаимодействия, если взаимодействие было, и стоит ноль, если взаимодействия не было.

In [17]:
from scipy.sparse import csr_matrix

def create_sparse_matrix(interactions_df, n_users, n_articles):    
    sparse_matrix = pd.DataFrame(0, index=n_users, columns=n_articles)
    
    
    user_ids = interactions_df['personId'].values
    content_ids = interactions_df['contentId'].values
    eventStrengths = interactions_df['eventStrength'].values
    
    
    for i in range(len(interactions_df)):
        sparse_matrix.loc[user_ids[i], content_ids[i]] = eventStrengths[i] if eventStrengths[i] is not np.nan else 0
    
    return csr_matrix(sparse_matrix.values)

In [18]:
n_users = np.unique(interactions_full_df['personId'])
n_articles = np.unique(interactions_full_df['contentId'])

In [19]:
data_train = create_sparse_matrix(interactions_train_df, n_users, n_articles)
data_test = create_sparse_matrix(interactions_test_df, n_users, n_articles)

## Задание 2 (0.5 балла)

Обучите модель LightFM с `loss="warp"` и посчитайте *precision@10* на тесте.

In [20]:
model = LightFM(loss='warp', random_state=42)
model.fit(data_train, epochs=20);

In [21]:
precision_at_k(model, data_test, data_train, k=10).mean()

0.0058044805

Низкое качество

## Задание 3 (2 балла)

При вызове метода `fit` LightFM позволяет передавать в `item_features` признаковое описание объектов. Воспользуемся этим. Будем получать признаковое описание из текста статьи в виде [TF-IDF](https://ru.wikipedia.org/wiki/TF-IDF) (можно воспользоваться `TfidfVectorizer` из scikit-learn). Создайте матрицу `feat` размером количесвто статей на размер признакового описание и обучите LightFM с `loss="warp"` и посчитайте precision@10 на тесте.

In [22]:
# Склеим текст и заголовок
feat = articles_df[['title', 'text']]
feat.index = articles_df['contentId']
feat = feat.reindex(n_articles).fillna('')
feat = feat.apply(lambda x: x['title'] + ' ' + x['text'], axis=1)
feat


contentId
-1006791494035379303    Google unleashes DeepMind on energy-hungry dat...
-1021685224930603833    Indústria 4.0: desafios e oportunidades *Igor ...
-1022885988494278200    12 JavaScript Hacks In this post I will share ...
-1024046541613287684    Australian Bitcoin Entrepreneur Launches Robo-...
-1033806831489252007    React Native v0.32.0-rc.0 released v0.32.0-rc....
                                              ...                        
967143806332397325      Baidu abre laboratório de realidade aumentada ...
972258375127367383      Better Exposed Filters  The Better Exposed Fil...
980458131533897249      Elasticsearch: CSV exporter for Kibana Discove...
98528655405030624       Quer reclamar? Desenvolvedores vencem hackatho...
991271693336573226      Este é o melhor jeito de entender como as mulh...
Length: 2984, dtype: object

In [23]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()

# учимся только на статьях из трейна
vectorizer.fit(feat.reindex(interactions_train_df['contentId'].unique()).fillna(''))

feat_tranform = vectorizer.transform(feat)

In [24]:
model.fit(data_train, item_features=feat_tranform, epochs=20);

In [ ]:
precision_at_k(model, data_test, data_train, k=10, item_features=feat_tranform).mean()

0.006822811

Качество улучшилось

## Задание 4 (1.5 балла)

В задании 3 мы использовали сырой текст статей. В этом задании необходимо сначала сделать предобработку текста (привести к нижнему регистру, убрать стоп слова, привести слова к номральной форме и т.д.), после чего обучите модель и оценить качество на тестовых данных.

In [26]:
!pip install pymorphy2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 KB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 58.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13721 sha256=59b80a338f3dec228bdb9b6f0531f4cd43f50a61e69609630f36d987b39fd0ff
  Stored in directory: /root/.cache/pip/wheels/70/4a/46/1309fc853b8d395e60bafaf1b6df7845bdd82c95fd59dd8d2b
Successfully built docopt


In [53]:
import regex
from pymorphy2 import MorphAnalyzer
from functools import lru_cache
from nltk.corpus import stopwords

import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

Посмотрим какие есть языки

In [28]:
np.unique(articles_df['lang'], return_counts=True)

(array(['en', 'es', 'ja', 'la', 'pt'], dtype=object),
 array([2211,    2,    2,    3,  829]))

In [29]:
stopwords.fileids()

['arabic',
 'azerbaijani',
 'basque',
 'bengali',
 'catalan',
 'chinese',
 'danish',
 'dutch',
 'english',
 'finnish',
 'french',
 'german',
 'greek',
 'hebrew',
 'hinglish',
 'hungarian',
 'indonesian',
 'italian',
 'kazakh',
 'nepali',
 'norwegian',
 'portuguese',
 'romanian',
 'russian',
 'slovene',
 'spanish',
 'swedish',
 'tajik',
 'turkish']

nltk предлагает возможность взять готовый набор стоп-слов для английского, португальского и испанского. Латынь и японский за неимением готового набора стоп-слов и малым количеством наблюдений остаются без стоп слов и очищаются только от пунктуации.

In [62]:
m = MorphAnalyzer()

def words_only(text):
    try:
        return regex.findall(r'[\p{L}]+', text.lower())
    except:
        return []

@lru_cache(maxsize=128)
def lemmatize_word(token, pymorphy=m):
    return pymorphy.parse(token)[0].normal_form

def lemmatize_text(text):
    return [lemmatize_word(w) for w in text]

def remove_stopwords(lemmas, stopwords):
    return [w for w in lemmas if not w in stopwords]

def clean_text(text, stopwords):
    tokens = words_only(text)
    lemmas = lemmatize_text(tokens)
    
    return ' '.join(remove_stopwords(lemmas, stopwords))

In [63]:
en_stopwords = stopwords.words('english')
pt_stopwords = stopwords.words('portuguese')  
es_stopwords = stopwords.words('spanish')

In [64]:
process_feat = {}

for key in feat.keys():
    info = articles_df[articles_df['contentId'] == key].values

    if (len(info) == 0):
      process_feat[key] = ''
      continue

    lang = info[0][-1]

    if lang == 'en':
      process_feat[key] = clean_text(feat[key], en_stopwords)
    elif lang == 'pt':
      process_feat[key] = clean_text(feat[key], pt_stopwords)
    elif lang == 'es':
      process_feat[key] = clean_text(feat[key], es_stopwords)
    else:
      process_feat[key] = clean_text(feat[key], [])


process_feat = pd.Series(process_feat)
process_feat.index.name = 'contentId'

In [65]:
process_feat

contentId
-1006791494035379303    google unleashes deepmind energy hungry datace...
-1021685224930603833    indústria desafios oportunidades igor schiewig...
-1022885988494278200    javascript hacks post share extremely useful h...
-1024046541613287684    australian bitcoin entrepreneur launches robo ...
-1033806831489252007    react native v rc released v rc github npm bre...
                                              ...                        
967143806332397325      baidu abre laboratório realidade aumentada sta...
972258375127367383      better exposed filters better exposed filters ...
980458131533897249      elasticsearch csv exporter kibana discover use...
98528655405030624       quer reclamar desenvolvedores vencem hackathon...
991271693336573226      melhor jeito entender mulheres sofrem machismo...
Length: 2984, dtype: object

In [66]:
# учимся только на статьях из трейна
vectorizer.fit(process_feat.reindex(interactions_train_df['contentId'].unique()).fillna(''))

process_feat_tranform = vectorizer.transform(process_feat)

In [76]:
model.fit(data_train, item_features=process_feat_tranform, epochs=20);

In [77]:
precision_at_k(model, data_test, data_train, k=10, item_features=process_feat_tranform).mean()

0.0069246436

In [ ]:
precision_at_k(model, data_test, data_train, k=10, item_features=process_feat_tranform).mean()

0.007739308

Улучшилось ли качество предсказания?

Качество улучшилось

## Задание 5 (1.5 балла)

Подберите гиперпараметры модели LightFM (`n_components` и др.) для улучшения качества модели.

In [72]:
!pip install bayesian-optimization

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [73]:
from bayes_opt import BayesianOptimization

parameters = {
    'components_num': (20, 80),
    'epoch_num': (5, 50),
    'learning_rate': (0.01, 0.1)
}


def optimisation_func(components_num, epoch_num, learning_rate):
    model = LightFM(learning_rate=learning_rate, loss='warp',
                    no_components=int(components_num))
    model.fit(data_train, item_features=process_feat_tranform,
              epochs=int(epoch_num))

    precision = precision_at_k(model, data_test, data_train,
                               k=10, item_features=process_feat_tranform).mean()
    
    return precision
  
optimizer = BayesianOptimization(
  f = optimisation_func,
  pbounds = parameters,
  verbose = 5,
  random_state = 42, 
)

optimizer.maximize(
  init_points = 4,
  n_iter = 3, 
)

|   iter    |  target   | compon... | epoch_num | learni... |
-------------------------------------------------------------
| 1         | 0.005804  | 42.47     | 47.78     | 0.07588   |
| 2         | 0.006721  | 55.92     | 12.02     | 0.02404   |
| 3         | 0.007332  | 23.49     | 43.98     | 0.0641    |
| 4         | 0.005804  | 62.48     | 5.926     | 0.09729   |
| 5         | 0.007026  | 23.84     | 43.81     | 0.09653   |
| 6         | 0.007026  | 22.43     | 43.6      | 0.01249   |


KeyboardInterrupt: ignored

Попробуем обучить на лучшем результате 

| 3         | 0.007332  | 23.49     | 43.98     | 0.0641    

In [81]:
best_model = LightFM(learning_rate=0.06, loss='warp', no_components=23)
best_model.fit(data_train, item_features=process_feat_tranform, epochs=44);

In [82]:
precision_at_k(best_model, data_test, data_train, k=10, item_features=process_feat_tranform).mean()

0.007942974

## Задание 6 (1 балл)

Реализуйте функции для вычисления следующих метрик:
* precision@k
* recall@k
* NDCG@k



In [ ]:
# Ваш код здесь

## Задание 7 (1 балл)

Вычислите значения реализованных метрик для $k=10$ для лучшей полученной модели в предыдущих шагах.

Найдите уже реализованные варианты этих метрик в библиотеках lightfm и sklearn. Сравните полученные у вас значения метрик с результатами встроенных в библиотеки метрик.

In [ ]:
# Ваш код здесь

## Задание 8 (1 балл)

Реализуйте алгоритм ALS и примените его для решения задачи ноутбука.

**ALS**

Итак, поставлена задача построения модели со скрытыми переменными (latent factor model) для коллаборативной фильтрации:

$$ \sum_{u,i} (r_{ui} - \langle p_u, q_i \rangle)^2 \to \min_{P,Q}$$

Суммирование ведется по всем парам $(u, i),$ для которых известен рейтинг $r_{ui}$ (и только по ним), а $p_u, q_i$ – латентные представления пользователя~$u$ и товара $i$, соответственно, матрицы $P, Q$ получаются путем записывания по столбцам векторов $p_u, q_i$ соответственно.

Подход ALS (Alternating Least Squares) решает задачу, попеременно фиксируя матрицы $P$ и $Q$, — оказывается, что, зафиксировав одну из матриц, можно выписать аналитическое решение задачи для другой.

$$\nabla_{p_u} \bigg[ \sum_{u,i} (r_{ui} - \langle p_u, q_i \rangle)^2 \bigg] = \sum_{i} 2(r_{ui} - \langle p_u, q_i \rangle)q_i = 0$$

Воспользовавшись тем, что $a^Tbc = cb^Ta$, получим
$$\sum_{i} r_{ui}q_i - \sum_i q_i q_i^T p_u = 0.$$

Тогда окончательно каждый столбец матрицы $P$ можно найти по формуле
$$p_u = \bigg( \sum_i q_i q_i^T\bigg)^{-1}\sum_ir_{ui}q_i \;\; \forall u,$$

аналогично для столбцов матрицы $Q$
$$q_i = \bigg( \sum_u p_u p_u^T\bigg)^{-1}\sum_ur_{ui}p_u \;\; \forall i.$$

Таким образом мы можем решать оптимизационную задачу, поочередно фиксируя одну из матриц $P$ или $Q$ и проводя оптимизацию по второй.

**Оригинальная статья c постановкой задачи для ALS на explicit feedback:**

* Bell, R.M. and Koren, Y., 2007, October. Scalable collaborative filtering with jointly derived neighborhood interpolation weights. In Seventh IEEE international conference on data mining (ICDM 2007) (pp. 43-52). IEEE.

**Оригинальная статья с ALS для implicit данных, которая стала более известной:**

* Hu, Y., Koren, Y. and Volinsky, C., 2008, December. Collaborative filtering for implicit feedback datasets. In 2008 Eighth IEEE international conference on data mining (pp. 263-272). Ieee.


In [ ]:
# Ваш код здесь